# 环境配置

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from tqdm import trange

# The multivariate Variance Gamma model basket option pricing and calibration 论文复刻

## Parameter

$r = 3\%$  
$\rho_{i,j}=0, ~for~i \ne j ~and~ i, j = 1, 2, 3$  
$N = 10^6$

| 1| 1 | 2 | 3|
|- |- |- |- |
|$\mu$|-0.1368 | -0.056 | -0.1984 |
|$\sigma$| 0.1099 | 0.1677 | 0.0365 |
| $X(0)$ | 100 | 200 | 300 |
| $\omega$ | 1/3 | 1/6 | 1/9 |

$$\varepsilon[K]=\frac{|\overline{C}[K] - C^{sim}[K]|}{C^{sim}[K]}$$

In [2]:
r = 0.03
rho = np.eye(3)
sim_times = 10 ** 6

## table1

In [12]:
X_0 = np.array([100, 200, 300])
q = np.array([0, 0, 0])
omega = np.array([0, 0, 0])
mu = np.array([-0.1368, -0.056, -0.1984])
sigma = np.array([0.1099, 0.1677, 0.0365])
weight = np.array([1/3, 1/6, 1/9]) # w是权重，omega才是参数
K = np.array([0, 0, 0])
parameter_list = [X_0, q, omega, mu, sigma, w, K]

table1 = pd.DataFrame(data =  {'X_0': X_0, 'mu': mu,'sigma': sigma, 'weight' : weight},
                      index =  ['stock 1', 'stock 2', 'stock 3'])
table1

,X_0,mu,sigma,weight
stock 1,100,-0.1368,0.1099,0.333333
stock 2,200,-0.0560,0.1677,0.166667
stock 3,300,-0.1984,0.0365,0.111111


## table2

先做个表头

In [331]:
from itertools import product
# T, v, K, C_hat, C_sim, vare

T = [1, 2]
v = [0.5, 0.75, 0.9]
K = [75, 90, 100, 110, 125]

table2 = pd.DataFrame(list(product(T, v, K)), columns = ['T', 'v', 'K']).set_index(['T','v', 'K'])
table2.loc[:,['$\overline{C}[K]$','$C^sim[K]$', r'$\varepsilon[K]$']] = 0
table2.T

T                    1                                               ...    2  \
v                 0.50                     0.75                      ... 0.75   
K                  75   90   100  110  125  75   90   100  110  125  ...  75    
$\overline{C}[K]$  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
$C^sim[K]$         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
$\varepsilon[K]$   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

T                                                               
v                                     0.90                      
K                  90   100  110  125  75   90   100  110  125  
$\overline{C}[K]$  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
$C^sim[K]$         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
$\varepsilon[K]$   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[3 rows x 30 columns]

### 计算所有T, v, K

In [332]:
from scipy.stats import norm
import scipy.integrate as sci
import math
from scipy.stats import norm
from scipy.optimize import fsolve # 解决非线性方程组的数值求解问题
from scipy.optimize import bisect # 解决非线性方程组的数值求解问题
from scipy.stats import gamma # gamma分布的概率密度

#### upper

In [333]:
from tqdm import tqdm
C_upper_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    # T, v, K = table2.index[i] # 1/6, 0.5, 225
    # print(T, v, K, r, sim_times)
    omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
    K_i = K / w.copy() / len(table1)
    # 通过奇怪的K解决了解析式的问题...K_i和omega点乘=K就可以了！
    def f(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        d_1 = (np.log(X_0/K_i) + (r - q + omega) * T + mu * x + sigma ** 2 * x / 2)/(sigma * np.sqrt(x))
        d_2 = d_1 - sigma * np.sqrt(x)
        exp_upper = np.exp((omega - q) * T + (mu + sigma ** 2 / 2) * x)
        X_price_numerical = ((X_0 * exp_upper * norm.cdf(d_1) - K_i * np.exp(-r * T) * norm.cdf(d_2)))
        S_price_numerical = sum(X_price_numerical * weight)
        return max(S_price_numerical, 0) * gamma_density
    C_upper = sci.quad(f, 0, np.inf)[0]
    C_upper_lst.append(C_upper)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:04<00:00,  6.41it/s]


In [334]:
table2.loc[:,'$C^u$'] = C_upper_lst
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$      $C^u$
T v    K                                                              
1 0.50 75                 0.0         0.0               0.0  27.540917
       90                 0.0         0.0               0.0  14.423531
       100                0.0         0.0               0.0   7.365173
       110                0.0         0.0               0.0   2.683204
       125                0.0         0.0               0.0   0.383341
  0.75 75                 0.0         0.0               0.0  27.749003
       90                 0.0         0.0               0.0  14.829547
       100                0.0         0.0               0.0   7.760264
       110                0.0         0.0               0.0   2.876199
       125                0.0         0.0               0.0   0.363491
  0.90 75                 0.0         0.0               0.0  27.873859
       90                 0.0         0.0               0.0  15.037197
       100                0.0         0.0               0.0   7.948567
       110                0.0         0.0               0.0   2.961497
       125                0.0         0.0               0.0   0.353159
2 0.50 75                 0.0         0.0               0.0  30.083968
       90                 0.0         0.0               0.0  18.005882
       100                0.0         0.0               0.0  11.440970
       110                0.0         0.0               0.0   6.478317
       125                0.0         0.0               0.0   2.155446
  0.75 75                 0.0         0.0               0.0  30.426178
       90                 0.0         0.0               0.0  18.586346
       100                0.0         0.0               0.0  12.073921
       110                0.0         0.0               0.0   7.025753
       125                0.0         0.0               0.0   2.402242
  0.90 75                 0.0         0.0               0.0  30.625656
       90                 0.0         0.0               0.0  18.887935
       100                0.0         0.0               0.0  12.388033
       110                0.0         0.0               0.0   7.290965
       125                0.0         0.0               0.0   2.525648

#### lower

权重是$w$  
$$\lambda_i = w_j X_j(0)\exp\{(r-q_j+\omega_j)T+\mu_j y+\frac{\sigma_j^2 y}{2}\}$$  
$$\sigma_{\Lambda_y}^2=\sum_{i=1}^n\lambda_i^2\sigma_i^2+\sum_{i=1, i\ne j}^n \lambda_i \lambda_j \sigma_i \sigma_j \rho_{i,j}$$  
$$r_i=\frac{\sum_{i=1}^n\lambda_j\sigma_j\rho_{i,j}}{\sigma_{\Lambda_y}}$$

In [335]:
from tqdm import tqdm
C_lower_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    # T, v, K = table2.index[i] # 1/6, 0.5, 225
    # print(T, v, K, r, sim_times)
    # 通过奇怪的K解决了解析式的问题...K_i和omega点乘=K就可以了！
    def g(x):
        omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
        K_i = K / weight.copy() / len(table1)
        lambda_i = weight * X_0 * np.exp((r - q + omega) * T + (mu + sigma ** 2 / 2) * x)
        sigma2Lambda = np.dot(np.dot((lambda_i * sigma),rho),(lambda_i * sigma)) # 是个矩阵
        r_i = np.dot((lambda_i * sigma),rho)/sigma2Lambda
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        K_i = K_i
        d_1 = (np.log(X_0/K_i) + (r - q + omega) * T + mu * x + sigma ** 2 * x / 2 * (1 + r_i ** 2))/(sigma * np.sqrt(x) * r_i)
        d_2 = d_1 - sigma * np.sqrt(x) * r_i
        exp_lower = np.exp((omega - q) * T + (mu + sigma ** 2 / 2) * x)
        X_price_numerical = ((X_0 * exp_lower * norm.cdf(d_1) - K_i * np.exp(-r * T) * norm.cdf(d_2)))
        S_price_numerical = sum(X_price_numerical * weight)
        return max(S_price_numerical, 0) * gamma_density
    C_lower = sci.quad(g, 0, np.inf)[0]
    C_lower_lst.append(C_lower)

100%|██████████| 30/30 [00:08<00:00,  3.70it/s]


In [336]:
table2.loc[:,'$C^l$'] = C_lower_lst
table2 = table2.round(4)
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$    $C^u$    $C^l$
T v    K                                                                     
1 0.50 75                 0.0         0.0               0.0  27.5409  27.3232
       90                 0.0         0.0               0.0  14.4235  13.3746
       100                0.0         0.0               0.0   7.3652   5.2373
       110                0.0         0.0               0.0   2.6832   0.8673
       125                0.0         0.0               0.0   0.3833   0.0000
  0.75 75                 0.0         0.0               0.0  27.7490  27.4721
       90                 0.0         0.0               0.0  14.8295  13.7845
       100                0.0         0.0               0.0   7.7603   5.8148
       110                0.0         0.0               0.0   2.8762   1.1914
       125                0.0         0.0               0.0   0.3635   0.0000
  0.90 75                 0.0         0.0               0.0  27.8739  27.5708
       90                 0.0         0.0               0.0  15.0372  14.0015
       100                0.0         0.0               0.0   7.9486   6.0911
       110                0.0         0.0               0.0   2.9615   1.3453
       125                0.0         0.0               0.0   0.3532   0.0000
2 0.50 75                 0.0         0.0               0.0  30.0840  29.5989
       90                 0.0         0.0               0.0  18.0059  16.3435
       100                0.0         0.0               0.0  11.4410   8.5573
       110                0.0         0.0               0.0   6.4783   3.0927
       125                0.0         0.0               0.0   2.1554   0.4562
  0.75 75                 0.0         0.0               0.0  30.4262  29.8647
       90                 0.0         0.0               0.0  18.5863  16.9462
       100                0.0         0.0               0.0  12.0739   9.3881
       110                0.0         0.0               0.0   7.0258   3.8755
       125                0.0         0.0               0.0   2.4022   0.7564
  0.90 75                 0.0         0.0               0.0  30.6257  30.0322
       90                 0.0         0.0               0.0  18.8879  17.2668
       100                0.0         0.0               0.0  12.3880   9.7996
       110                0.0         0.0               0.0   7.2910   4.2558
       125                0.0         0.0               0.0   2.5256   0.9164

#### z_y to approximate

##### var upper

\begin{align}
& \operatorname{Var}\left[S_y^c\right]=\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) \\
& \quad e^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j\right) y+\frac{\sigma_i^2+\sigma_j^2}{2} y}\left(e^{\sigma_i \sigma_j y}-1\right) .
\end{align}

In [337]:
from tqdm import tqdm
var_upper_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    def var_upper_calculate(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_upper_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_upper_part2 = np.exp(2 * r * T)
        var_upper_part3 = np.outer(np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x),np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x))
        var_upper_part4 = np.exp(np.outer(sigma,sigma) * x) - 1
        var_upper_int = sum(sum(var_upper_part1 * var_upper_part2 * var_upper_part3 * var_upper_part4))
        return var_upper_int * gamma_density
    var_upper = sci.quad(var_upper_calculate, 0, np.inf)[0]
    var_upper_lst.append(var_upper)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:02<00:00, 11.17it/s]


In [338]:
table2.loc[:,'$Var^u$'] = var_upper_lst
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$    $C^u$    $C^l$  \
T v    K                                                                        
1 0.50 75                 0.0         0.0               0.0  27.5409  27.3232   
       90                 0.0         0.0               0.0  14.4235  13.3746   
       100                0.0         0.0               0.0   7.3652   5.2373   
       110                0.0         0.0               0.0   2.6832   0.8673   
       125                0.0         0.0               0.0   0.3833   0.0000   
  0.75 75                 0.0         0.0               0.0  27.7490  27.4721   
       90                 0.0         0.0               0.0  14.8295  13.7845   
       100                0.0         0.0               0.0   7.7603   5.8148   
       110                0.0         0.0               0.0   2.8762   1.1914   
       125                0.0         0.0               0.0   0.3635   0.0000   
  0.90 75                 0.0         0.0               0.0  27.8739  27.5708   
       90                 0.0         0.0               0.0  15.0372  14.0015   
       100                0.0         0.0               0.0   7.9486   6.0911   
       110                0.0         0.0               0.0   2.9615   1.3453   
       125                0.0         0.0               0.0   0.3532   0.0000   
2 0.50 75                 0.0         0.0               0.0  30.0840  29.5989   
       90                 0.0         0.0               0.0  18.0059  16.3435   
       100                0.0         0.0               0.0  11.4410   8.5573   
       110                0.0         0.0               0.0   6.4783   3.0927   
       125                0.0         0.0               0.0   2.1554   0.4562   
  0.75 75                 0.0         0.0               0.0  30.4262  29.8647   
       90                 0.0         0.0               0.0  18.5863  16.9462   
       100                0.0         0.0               0.0  12.0739   9.3881   
       110                0.0         0.0               0.0   7.0258   3.8755   
       125                0.0         0.0               0.0   2.4022   0.7564   
  0.90 75                 0.0         0.0               0.0  30.6257  30.0322   
       90                 0.0         0.0               0.0  18.8879  17.2668   
       100                0.0         0.0               0.0  12.3880   9.7996   
       110                0.0         0.0               0.0   7.2910   4.2558   
       125                0.0         0.0               0.0   2.5256   0.9164   

               $Var^u$  
T v    K                
1 0.50 75   141.934946  
       90   141.934946  
       100  141.934946  
       110  141.934946  
       125  141.934946  
  0.75 75   137.247288  
       90   137.247288  
       100  137.247288  
       110  137.247288  
       125  137.247288  
  0.90 75   134.607742  
       90   134.607742  
       100  134.607742  
       110  134.607742  
       125  134.607742  
2 0.50 75   403.716205  
       90   403.716205  
       100  403.716205  
       110  403.716205  
       125  403.716205  
  0.75 75   391.803477  
       90   391.803477  
       100  391.803477  
       110  391.803477  
       125  391.803477  
  0.90 75   385.050026  
       90   385.050026  
       100  385.050026  
       110  385.050026  
       125  385.050026

\begin{align*}
\operatorname{Var} & {\left[S_y^l\right]=\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) } \\
& \times \mathrm{e}^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j+\frac{1}{2}\left(\sigma_i^2\left(1-r_i^2\right)+\sigma_j^2\left(1-r_j^2\right)\right)\right) y} \\
& \times e^{\frac{1}{2}(r^2_i\sigma^2_i+r^2_j\sigma^2_j)y}(e^{r_ir_j\sigma_i\sigma_jy}-1)\\
\end{align*}

\begin{align}
& {=\sum_{i=1}^n \sum_{j=1}^n w_i w_j X_i(0) X_j(0) } \\
& \times \mathrm{e}^{2 r T+\left(\omega_i-q_i+\omega_j-q_j\right) T+\left(\mu_i+\mu_j+\frac{1}{2}\left(\sigma_i^2+\sigma_j^2\right)\right) y} \\
& \times (e^{r_ir_j\sigma_i\sigma_jy}-1)
\end{align}

权重是$w$  
$$\lambda_i = w_j X_j(0)\exp\{(r-q_j+\omega_j)T+\mu_j y+\frac{\sigma_j^2 y}{2}\}$$  
$$\sigma_{\Lambda_y}^2=\sum_{i=1}^n\lambda_i^2\sigma_i^2+\sum_{i=1, i\ne j}^n \lambda_i \lambda_j \sigma_i \sigma_j \rho_{i,j}$$  
$$r_i=\frac{\sum_{i=1}^n\lambda_j\sigma_j\rho_{i,j}}{\sigma_{\Lambda_y}}$$

##### var lower 

In [339]:
from tqdm import tqdm
var_lower_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    def var_lower_calculation(x):
        lambda_i = weight * X_0 * np.exp((r - q + omega) * T + (mu + sigma ** 2 / 2) * x)
        sigma2Lambda = np.dot(np.dot((lambda_i * sigma),rho),(lambda_i * sigma)) # 是个矩阵
        r_i = np.dot((lambda_i * sigma),rho)/np.sqrt(sigma2Lambda)
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_lower_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_lower_part2 = np.exp(2 * r * T)
        var_lower_part3 = np.outer(np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x),np.exp((weight - q) * T + mu * x + sigma ** 2 / 2 * x))
        var_lower_part4 = np.exp(np.outer(sigma * r_i, sigma * r_i) * x) - 1
        var_lower_int = sum(sum(var_lower_part1 * var_lower_part2 * var_lower_part3 * var_lower_part4))
        return var_lower_int * gamma_density
    var_lower = sci.quad(var_lower_calculation, 0, np.inf)[0]
    var_lower_lst.append(var_lower)

100%|██████████| 30/30 [00:02<00:00, 10.54it/s]


In [340]:
table2.loc[:,'$Var^l$'] = var_lower_lst
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$    $C^u$    $C^l$  \
T v    K                                                                        
1 0.50 75                 0.0         0.0               0.0  27.5409  27.3232   
       90                 0.0         0.0               0.0  14.4235  13.3746   
       100                0.0         0.0               0.0   7.3652   5.2373   
       110                0.0         0.0               0.0   2.6832   0.8673   
       125                0.0         0.0               0.0   0.3833   0.0000   
  0.75 75                 0.0         0.0               0.0  27.7490  27.4721   
       90                 0.0         0.0               0.0  14.8295  13.7845   
       100                0.0         0.0               0.0   7.7603   5.8148   
       110                0.0         0.0               0.0   2.8762   1.1914   
       125                0.0         0.0               0.0   0.3635   0.0000   
  0.90 75                 0.0         0.0               0.0  27.8739  27.5708   
       90                 0.0         0.0               0.0  15.0372  14.0015   
       100                0.0         0.0               0.0   7.9486   6.0911   
       110                0.0         0.0               0.0   2.9615   1.3453   
       125                0.0         0.0               0.0   0.3532   0.0000   
2 0.50 75                 0.0         0.0               0.0  30.0840  29.5989   
       90                 0.0         0.0               0.0  18.0059  16.3435   
       100                0.0         0.0               0.0  11.4410   8.5573   
       110                0.0         0.0               0.0   6.4783   3.0927   
       125                0.0         0.0               0.0   2.1554   0.4562   
  0.75 75                 0.0         0.0               0.0  30.4262  29.8647   
       90                 0.0         0.0               0.0  18.5863  16.9462   
       100                0.0         0.0               0.0  12.0739   9.3881   
       110                0.0         0.0               0.0   7.0258   3.8755   
       125                0.0         0.0               0.0   2.4022   0.7564   
  0.90 75                 0.0         0.0               0.0  30.6257  30.0322   
       90                 0.0         0.0               0.0  18.8879  17.2668   
       100                0.0         0.0               0.0  12.3880   9.7996   
       110                0.0         0.0               0.0   7.2910   4.2558   
       125                0.0         0.0               0.0   2.5256   0.9164   

               $Var^u$     $Var^l$  
T v    K                            
1 0.50 75   141.934946   61.694932  
       90   141.934946   61.694932  
       100  141.934946   61.694932  
       110  141.934946   61.694932  
       125  141.934946   61.694932  
  0.75 75   137.247288   60.017272  
       90   137.247288   60.017272  
       100  137.247288   60.017272  
       110  137.247288   60.017272  
       125  137.247288   60.017272  
  0.90 75   134.607742   59.070035  
       90   134.607742   59.070035  
       100  134.607742   59.070035  
       110  134.607742   59.070035  
       125  134.607742   59.070035  
2 0.50 75   403.716205  176.971319  
       90   403.716205  176.971319  
       100  403.716205  176.971319  
       110  403.716205  176.971319  
       125  403.716205  176.971319  
  0.75 75   391.803477  172.553123  
       90   391.803477  172.553123  
       100  391.803477  172.553123  
       110  391.803477  172.553123  
       125  391.803477  172.553123  
  0.90 75   385.050026  170.043247  
       90   385.050026  170.043247  
       100  385.050026  170.043247  
       110  385.050026  170.043247  
       125  385.050026  170.043247

##### var Sy

\begin{align*}
	\mathrm{Var}[{_y}S]=\sum_{i=1}^{n} \sum_{j=1}^{n} w_{i} w_{j} X_{i}(0) X_{j}(0)  
	\mathrm{e}^{2 r T+\left(\omega_{i}-q_{i}+\omega_{j}-q_{j}\right) T+\left(\mu_{i}+\mu_{j}\right) y} 
	e^{\frac{\sigma_i^2+\sigma_j^2+\rho_{i,j}\sigma_i\sigma_j
	}{4-\rho^2_{i,j}}2(1-\rho^2_{i,j})y}
\end{align*}


In [341]:
from tqdm import tqdm
var_Sy_lst = list()
for (T,v,K) in tqdm(table2.index.values):
    def var_calculation(x):
        gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
        var_part1 = np.outer(X_0 * weight, X_0 * weight)
        var_part2 = np.exp(2 * r * T)
        var_part3 = np.outer(np.exp((weight - q) * T + mu * x),np.exp((weight - q) * T + mu * x))
        var_part4_i_ne_j = np.exp((sigma ** 2 + (sigma ** 2).reshape(len(sigma),-1) + np.outer(sigma, sigma) * rho) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))
        np.fill_diagonal(var_part4_i_ne_j, np.outer(np.exp(x * sigma ** 2), np.exp(x * sigma ** 2))) # 生成E[XY] # TODO
        var_part4 = var_part4_i_ne_j - np.outer(np.exp(sigma ** 2 / 2 * x), np.exp(sigma ** 2 / 2 * x))
        # var_part4 = (np.exp((sigma ** 2) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))) * (np.exp((sigma ** 2) * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))) #* np.exp(np.outer(sigma,sigma) * rho  * 2 * (1 - rho ** 2) * x/(4 - rho ** 2))
        var_int = sum(sum(var_part1 * var_part2 * var_part3 * var_part4))
        return var_int * gamma_density
    var_Sy = sci.quad(var_calculation, 0, np.inf)[0]
    var_Sy_lst.append(var_Sy)

100%|██████████| 30/30 [00:02<00:00, 10.97it/s]


In [342]:
table2.loc[:,'$Var[Sy]$'] = var_Sy_lst
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$    $C^u$    $C^l$  \
T v    K                                                                        
1 0.50 75                 0.0         0.0               0.0  27.5409  27.3232   
       90                 0.0         0.0               0.0  14.4235  13.3746   
       100                0.0         0.0               0.0   7.3652   5.2373   
       110                0.0         0.0               0.0   2.6832   0.8673   
       125                0.0         0.0               0.0   0.3833   0.0000   
  0.75 75                 0.0         0.0               0.0  27.7490  27.4721   
       90                 0.0         0.0               0.0  14.8295  13.7845   
       100                0.0         0.0               0.0   7.7603   5.8148   
       110                0.0         0.0               0.0   2.8762   1.1914   
       125                0.0         0.0               0.0   0.3635   0.0000   
  0.90 75                 0.0         0.0               0.0  27.8739  27.5708   
       90                 0.0         0.0               0.0  15.0372  14.0015   
       100                0.0         0.0               0.0   7.9486   6.0911   
       110                0.0         0.0               0.0   2.9615   1.3453   
       125                0.0         0.0               0.0   0.3532   0.0000   
2 0.50 75                 0.0         0.0               0.0  30.0840  29.5989   
       90                 0.0         0.0               0.0  18.0059  16.3435   
       100                0.0         0.0               0.0  11.4410   8.5573   
       110                0.0         0.0               0.0   6.4783   3.0927   
       125                0.0         0.0               0.0   2.1554   0.4562   
  0.75 75                 0.0         0.0               0.0  30.4262  29.8647   
       90                 0.0         0.0               0.0  18.5863  16.9462   
       100                0.0         0.0               0.0  12.0739   9.3881   
       110                0.0         0.0               0.0   7.0258   3.8755   
       125                0.0         0.0               0.0   2.4022   0.7564   
  0.90 75                 0.0         0.0               0.0  30.6257  30.0322   
       90                 0.0         0.0               0.0  18.8879  17.2668   
       100                0.0         0.0               0.0  12.3880   9.7996   
       110                0.0         0.0               0.0   7.2910   4.2558   
       125                0.0         0.0               0.0   2.5256   0.9164   

               $Var^u$     $Var^l$   $Var[Sy]$  
T v    K                                        
1 0.50 75   141.934946   61.694932   47.522901  
       90   141.934946   61.694932   47.522901  
       100  141.934946   61.694932   47.522901  
       110  141.934946   61.694932   47.522901  
       125  141.934946   61.694932   47.522901  
  0.75 75   137.247288   60.017272   45.646310  
       90   137.247288   60.017272   45.646310  
       100  137.247288   60.017272   45.646310  
       110  137.247288   60.017272   45.646310  
       125  137.247288   60.017272   45.646310  
  0.90 75   134.607742   59.070035   44.606960  
       90   134.607742   59.070035   44.606960  
       100  134.607742   59.070035   44.606960  
       110  134.607742   59.070035   44.606960  
       125  134.607742   59.070035   44.606960  
2 0.50 75   403.716205  176.971319  131.032311  
       90   403.716205  176.971319  131.032311  
       100  403.716205  176.971319  131.032311  
       110  403.716205  176.971319  131.032311  
       125  403.716205  176.971319  131.032311  
  0.75 75   391.803477  172.553123  126.688239  
       90   391.803477  172.553123  126.688239  
       100  391.803477  172.553123  126.688239  
       110  391.803477  172.553123  126.688239  
       125  391.803477  172.553123  126.688239  
  0.90 75   385.050026  170.043247  124.249679  
       90   385.050026  170.043247  124.249679 

$$z_y = \frac{Var[S_y^c]-Var[S_y]}{Var[S_y^c]-Var[S_y^l]}$$  

In [355]:
table2.loc[:,'$z_y$'] = (table2.loc[:,'$Var^u$'] - table2.loc[:,'$Var[Sy]$'])/(table2.loc[:,'$Var^u$'] - table2.loc[:,'$Var^l$'] ) - 1 # TODO
table2.loc[:,'$\overline{C}[K]$'] = table2.loc[:,'$z_y$'] * table2.loc[:,'$C^l$'] + (1-table2.loc[:,'$z_y$']) * table2.loc[:,'$C^u$']
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$    $C^u$    $C^l$  \
T v    K                                                                        
1 0.50 75           27.502450     27.3116          0.000983  27.5409  27.3232   
       90           14.238243     13.5774          0.028581  14.4235  13.3746   
       100           6.989369      5.9578          0.184016   7.3652   5.2373   
       110           2.362475      1.2027          0.545543   2.6832   0.8673   
       125           0.315601      0.0060         12.283106   0.3833   0.0000   
  0.75 75           27.697474     27.4386          0.000657  27.7490  27.4721   
       90           14.635046     13.9589          0.026424  14.8295  13.7845   
       100           7.398281      6.4220          0.150922   7.7603   5.8148   
       110           2.562692      1.3957          0.371002   2.8762   1.1914   
       125           0.295860      0.0051         14.262762   0.3635   0.0000   
  0.90 75           27.815866     27.5319          0.000695  27.8739  27.5708   
       90           14.838896     14.1706          0.025927  15.0372  14.0015   
       100           7.592948      6.6586          0.138641   7.9486   6.0911   
       110           2.652049      1.4925          0.305964   2.9615   1.3453   
       125           0.285573      0.0048         15.088875   0.3532   0.0000   
2 0.50 75           29.985718     29.6091          0.003664  30.0840  29.5989   
       90           17.669094     16.6890          0.040884  18.0059  16.3435   
       100          10.856756      9.4982          0.160572  11.4410   8.5573   
       110           5.792370      4.2383          0.432138   6.4783   3.0927   
       125           1.811138      0.5537          0.797836   2.1554   0.4562   
  0.75 75           30.308740     29.8354          0.002955  30.4262  29.8647   
       90           18.243208     17.2421          0.037060  18.5863  16.9462   
       100          11.512059     10.1977          0.134485  12.0739   9.3881   
       110           6.366790      4.8520          0.337080   7.0258   3.8755   
       125           2.057916      0.7026          0.413442   2.4022   0.7564   
  0.90 75           30.499292     29.9823          0.002552  30.6257  30.0322   
       90           18.542627     17.5383          0.035167  18.8879  17.2668   
       100          11.836705     10.5484          0.123250  12.3880   9.7996   
       110           6.644543      5.1560          0.299972   7.2910   4.2558   
       125           2.182862      0.7811          0.265572   2.5256   0.9164   

               $Var^u$     $Var^l$   $Var[Sy]$     $z_y$  
T v    K                                                  
1 0.50 75   141.934946   61.694932   47.522901  0.176620  
       90   141.934946   61.694932   47.522901  0.176620  
       100  141.934946   61.694932   47.522901  0.176620  
       110  141.934946   61.694932   47.522901  0.176620  
       125  141.934946   61.694932   47.522901  0.176620  
  0.75 75   137.247288   60.017272   45.646310  0.186080  
       90   137.247288   60.017272   45.646310  0.186080  
       100  137.247288   60.017272   45.646310  0.186080  
       110  137.247288   60.017272   45.646310  0.186080  
       125  137.247288   60.017272   45.646310  0.186080  
  0.90 75   134.607742   59.070035   44.606960  0.191468  
       90   134.607742   59.070035   44.606960  0.191468  
       100  134.607742   59.070035   44.606960  0.191468  
       110  134.607742   59.070035   44.606960  0.191468  
       125  134.607742   59.070035   44.606960  0.191468  
2 0.50 75   403.716205  176.971319  131.032311  0.202602  
       90   403.716205  176.971319  131.032311  0.202602  
       100  403.716205  176.971319  131.032311  0.202602  
       110  403.716205  176.971319  131.032311  0.202602  
       125  403.716205  176.971319  131.032311  0.202602  
  0.75 75   391.803477  172.553123  126.688239  0.209190  
       90   391.803477  172.553123  126.688239  0.209190  
   

#### sim

##### 生成随机数

##### 蒙特卡洛

##### For循环

In [344]:
from tqdm import tqdm
C_sim_list = list()
C_sim_result_list= list()
power_num = 3
for (T,v,K) in tqdm(table2.index.values):
    # T, v, K = table2.index[i] # 1/6, 0.5, 225
    # print(T, v, K, r, sim_times)
    # shape = at, scale = 1/b; v = 1/a = 1/b  ; shape = T/v, scale = v  
    np.random.seed(20240321) 
    Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=rho, size=sim_times)
    Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)
    omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
    K_i = K / weight.copy() / len(table1)
    for sim_num in range(10 ** power_num):
        X_price = X_0 * np.exp((r - q + omega) * T + mu * Gamma_values[sim_num] + sigma * np.sqrt(Gamma_values[sim_num]) * Normal_values[sim_num])
        S_price = sum(X_price * weight)
        C_sim_list.append(max(S_price - K, 0))
    C_sim = sum(C_sim_list) * np.exp(-r * T) / (10 ** power_num) # 原论文是27.3230
    C_sim_result_list.append(C_sim)
    C_sim_list = list()

100%|██████████| 30/30 [00:05<00:00,  5.29it/s]


##### 向量化（指令并行）

In [345]:
from tqdm import tqdm
C_sim_list = list()
C_sim_result_list= list()
power_num = 7
for (T,v,K) in tqdm(table2.index.values):
    # T, v, K = table2.index[i] # 1/6, 0.5, 225
    # print(T, v, K, r, sim_times)
    # shape = at, scale = 1/b; v = 1/a = 1/b  ; shape = T/v, scale = v  
    np.random.seed(20240321) 
    Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=rho, size=sim_times)
    Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)
    omega = 1/v * np.log(1 - 1/2 * (sigma ** 2) * v - mu * v)
    K_i = K / weight.copy() / len(table1)
    X_price_vector = X_0 * np.exp((r - q + omega) * T) * np.exp(np.outer(mu, Gamma_values)).T * np.exp(sigma * np.sqrt(Gamma_values.reshape(-1, 1)) * Normal_values)
    S_price_vector = np.dot(X_price_vector, weight)
    C_sim_vector_list = (S_price_vector - K) * (S_price_vector - K > 0)
    C_sim = sum(C_sim_vector_list) * np.exp(-r * T) / len(C_sim_vector_list) # 原论文是27.3230
    C_sim_result_list.append(C_sim)
    C_sim_list = list()

100%|██████████| 30/30 [00:09<00:00,  3.17it/s]


In [356]:
table2.loc[:,'$C^sim[K]$'] = np.round(C_sim_result_list,4)
table2.loc[:,r'$\varepsilon[K]$'] = abs(table2.loc[:,'$\overline{C}[K]$'] - table2.loc[:,'$C^sim[K]$'])/ table2.loc[:,'$C^sim[K]$']
table2

$\overline{C}[K]$  $C^sim[K]$  $\varepsilon[K]$    $C^u$    $C^l$  \
T v    K                                                                        
1 0.50 75           27.502450     27.3116          0.006988  27.5409  27.3232   
       90           14.238243     13.5774          0.048672  14.4235  13.3746   
       100           6.989369      5.9578          0.173146   7.3652   5.2373   
       110           2.362475      1.2027          0.964309   2.6832   0.8673   
       125           0.315601      0.0060         51.600228   0.3833   0.0000   
  0.75 75           27.697474     27.4386          0.009435  27.7490  27.4721   
       90           14.635046     13.9589          0.048438  14.8295  13.7845   
       100           7.398281      6.4220          0.152021   7.7603   5.8148   
       110           2.562692      1.3957          0.836134   2.8762   1.1914   
       125           0.295860      0.0051         57.011748   0.3635   0.0000   
  0.90 75           27.815866     27.5319          0.010314  27.8739  27.5708   
       90           14.838896     14.1706          0.047161  15.0372  14.0015   
       100           7.592948      6.6586          0.140322   7.9486   6.0911   
       110           2.652049      1.4925          0.776917   2.9615   1.3453   
       125           0.285573      0.0048         58.494459   0.3532   0.0000   
2 0.50 75           29.985718     29.6091          0.012720  30.0840  29.5989   
       90           17.669094     16.6890          0.058727  18.0059  16.3435   
       100          10.856756      9.4982          0.143033  11.4410   8.5573   
       110           5.792370      4.2383          0.366673   6.4783   3.0927   
       125           1.811138      0.5537          2.270974   2.1554   0.4562   
  0.75 75           30.308740     29.8354          0.015865  30.4262  29.8647   
       90           18.243208     17.2421          0.058062  18.5863  16.9462   
       100          11.512059     10.1977          0.128888  12.0739   9.3881   
       110           6.366790      4.8520          0.312199   7.0258   3.8755   
       125           2.057916      0.7026          1.929001   2.4022   0.7564   
  0.90 75           30.499292     29.9823          0.017243  30.6257  30.0322   
       90           18.542627     17.5383          0.057265  18.8879  17.2668   
       100          11.836705     10.5484          0.122133  12.3880   9.7996   
       110           6.644543      5.1560          0.288701   7.2910   4.2558   
       125           2.182862      0.7811          1.794600   2.5256   0.9164   

               $Var^u$     $Var^l$   $Var[Sy]$     $z_y$  
T v    K                                                  
1 0.50 75   141.934946   61.694932   47.522901  0.176620  
       90   141.934946   61.694932   47.522901  0.176620  
       100  141.934946   61.694932   47.522901  0.176620  
       110  141.934946   61.694932   47.522901  0.176620  
       125  141.934946   61.694932   47.522901  0.176620  
  0.75 75   137.247288   60.017272   45.646310  0.186080  
       90   137.247288   60.017272   45.646310  0.186080  
       100  137.247288   60.017272   45.646310  0.186080  
       110  137.247288   60.017272   45.646310  0.186080  
       125  137.247288   60.017272   45.646310  0.186080  
  0.90 75   134.607742   59.070035   44.606960  0.191468  
       90   134.607742   59.070035   44.606960  0.191468  
       100  134.607742   59.070035   44.606960  0.191468  
       110  134.607742   59.070035   44.606960  0.191468  
       125  134.607742   59.070035   44.606960  0.191468  
2 0.50 75   403.716205  176.971319  131.032311  0.202602  
       90   403.716205  176.971319  131.032311  0.202602  
       100  403.716205  176.971319  131.032311  0.202602  
       110  403.716205  176.971319  131.032311  0.202602  
       125  403.716205  176.971319  131.032311  0.202602  
  0.75 75   391.803477  172.553123  126.688239  0.209190  
       90   391.803477  172.553123  126.688239  0.209190  
   

# bak-代码备份

这是2014版论文里的参数，用这个模拟后和2014版论文也没能对上

In [ ]:
X_0 = [100.0, 100.0, 100.0]
q = [0, 0, 0]
w = [0, 0, 0]
mu = [-0.15, -0.06, -0.2]
sigma = [0.1, 0.2, 0.04]
omega = [1, 1, 1]

分段积分没啥用

In [ ]:
def g(x):
    return gamma(a=T/v, scale=v).pdf(x)
seperate_point1 = 2
print("# --- 计算第一段 --- #")
g_upper1 = sci.quad(g, 1e-16, seperate_point1)[0]
print(g_upper1)
print("# --- 计算第二段 --- #")
g_upper2 = sci.quad(g, seperate_point1, np.inf)[0]
print(g_upper2)
g_upper1 + g_upper2

# --- 计算第一段 --- #
0.9084218055563291
# --- 计算第二段 --- #
0.09157819444367087


1.0

In [ ]:
seperate_point1 = 0.3
seperate_point2 = 0.6
sci.quad(g, 0, seperate_point1)[0], sci.quad(g, seperate_point1, seperate_point2)[0], 

In [ ]:
i = 0
int_list = list()
for k in trange(100):
    int_list.append(sci.quad(f, i, i+0.05)[0])
    i += 0.05
    i = round(i,2)
int_list.append(sci.quad(f, i+0.05, np.inf)[0])
sum(int_list)

230623

In [ ]:
# 通过奇怪的K解决了解析式的问题，为什么K随便取呢
table1.loc[:,'K'] = [K, 2*K, 3*K]
def f(x):
    # gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
    gamma_density = ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
    # def S_yc(U):
    #     return sum(table1.loc[:,'w'] * table1.loc[:,'X_0'] * np.exp((r - table1.loc[:,'q'] + table1.loc[:,'omega']) * T + table1.loc[:,'mu'] * x + table1.loc[:,'sigma'] * np.sqrt(x) * norm.ppf(U)))
    # U_initial_guess = 1e-200
    # F_Sy = fsolve(lambda U: S_yc(U) - K, U_initial_guess)[0]
    # print(S_yc(F_Sy))
    # K_i = table1.loc[:,'X_0'] * np.exp((r - table1.loc[:,'q'] + table1.loc[:,'omega']) * T + table1.loc[:,'mu'] * x + table1.loc[:,'sigma'] * np.sqrt(x) * norm.ppf(F_Sy))
    K_i = table1.loc[:,'K']
    # return sum(K_i * table1.loc[:,'w'])
    d_1 = (np.log(table1.loc[:,'X_0']/K_i) + (r - table1.loc[:,'q'] + table1.loc[:,'omega']) * T + table1.loc[:,'mu'] * x + table1.loc[:,'sigma'] ** 2 * x)/(table1.loc[:,'sigma'] * np.sqrt(x))
    d_2 = d_1 - table1.loc[:,'sigma'] * np.sqrt(x)
    exp_upper = np.exp((table1.loc[:,'omega'] - table1.loc[:,'q']) * T + (table1.loc[:,'mu'] + table1.loc[:,'sigma'] ** 2 / 2) * x)
    X_price_numerical = ((table1.loc[:,'X_0'] * exp_upper * norm.cdf(d_1) - K_i * np.exp(-r * T) * norm.cdf(d_2)))
    S_price_numerical = sum(X_price_numerical * table1.loc[:,'w'])
    return max(S_price_numerical, 0) * gamma_density

## 230626 计算单组T, v, K

### 计算第一组T, v, K

#### 计算w

In [ ]:
i = 0
T, v, K = table2.index[i] # 1/6, 0.5, 225
print(T, v, K, r, sim_times)
table1.loc[:,'omega'] = 1/v * np.log(1 - 1/2 * (table1.loc[:,'sigma'] ** 2) * v - table1.loc[:,'mu'] * v)
table1.loc[:,'K'] = K / table1.loc[:,'w'].copy() / len(table1)
table1

1 0.5 75 0.03 1000000


,X_0,q,w,mu,sigma,omega,K
stock 1,100,0,0.126664,-0.1368,0.1099,0.333333,75.0
stock 2,200,0,0.041505,-0.0560,0.1677,0.166667,150.0
stock 3,300,0,0.188559,-0.1984,0.0365,0.111111,225.0


#### 生成随机数

shape = at, scale = 1/b; v = 1/a = 1/b  ; shape = T/v, scale = v  

In [ ]:
np.random.seed(20240321) 
Normal_values = multivariate_normal.rvs(mean=[0,0,0], cov=rho, size=sim_times)
Gamma_values = np.random.gamma(shape = T/v, scale = v, size=sim_times)
Normal_values, Gamma_values

(array([[-1.16407636, -0.68169192, -1.04735157],
        [-0.88232937, -2.18217986,  1.4435476 ],
        [ 0.30387315,  0.93289644,  0.75174888],
        ...,
        [ 1.48806088,  0.62710569,  1.31769641],
        [-0.46258561, -0.73427884,  1.19404285],
        [ 1.36406203,  0.65262665, -1.76303698]]),
 array([0.83237464, 0.46005296, 0.33989436, ..., 2.93328014, 2.06744345,
        0.6756765 ]))

#### 蒙特卡洛

In [ ]:
C_sim_list = list()
power_num = 4
for sim_num in trange(10 ** power_num):
    # X_price = table1.iloc[:,0] * np.exp((r - table1.iloc[:,1] + table1.iloc[:,2]) * T + table1.iloc[:,3] * Gamma_values[sim_num] + table1.iloc[:,4] * np.sqrt(Gamma_values[sim_num]) * Normal_values[sim_num])
    X_price = table1.loc[:,'X_0'] * np.exp((r - table1.loc[:,'q'] + table1.loc[:,'omega']) * T + table1.loc[:,'mu'] * Gamma_values[sim_num] + table1.loc[:,'sigma'] * np.sqrt(Gamma_values[sim_num]) * Normal_values[sim_num])
    S_price = sum(X_price * table1.loc[:,'w'])
    C_sim_list.append(max(S_price - K, 0))
C_sim = sum(C_sim_list) * np.exp(-r * T) / (10 ** power_num) # 原论文是27.3230
C_sim

100%|██████████| 10000/10000 [00:04<00:00, 2107.63it/s]


27.262810737972234

In [ ]:
# 看看和原文相比的误差
C_sim/27.3250 - 1

-0.0022759107786922694

#### 数值积分

在 `scipy.integrate` 里有五个数值积分的函数：

- `fixed_quad`：fixed Gaussian quadrature (定点高斯积分)
- `quad`：adaptive quadrature (自适应积分) # 取五个点，再五个点，随机波动率，heston等下算衍生品价格适合这个
- `romberg`：Romberg integration (龙贝格积分)
- `trapz`：用 trapezoidal 法则 # 矩形
- `simps`：用 Simpson’s 法则 # 梯形

前三个函数 `fixed_quad`, `quad`, `romberg` 有三个参数，分别是**被积函数**、**下界**和**上界**，使用它们的代码范式为

    func( f, lb, ub )

在下面实证中，除了`quad`外，积分结果都有点奇怪。

In [ ]:
from scipy.stats import norm
import scipy.integrate as sci
import math
from scipy.stats import norm
from scipy.optimize import fsolve # 解决非线性方程组的数值求解问题
from scipy.optimize import bisect # 解决非线性方程组的数值求解问题
from scipy.stats import gamma # gamma分布的概率密度

##### upper

In [ ]:
# 通过奇怪的K解决了解析式的问题...K_i和omega点乘=K就可以了！
def f(x):
    gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
    K_i = table1.loc[:,'K']
    d_1 = (np.log(table1.loc[:,'X_0']/K_i) + (r - table1.loc[:,'q'] + table1.loc[:,'omega']) * T + table1.loc[:,'mu'] * x + table1.loc[:,'sigma'] ** 2 * x)/(table1.loc[:,'sigma'] * np.sqrt(x))
    d_2 = d_1 - table1.loc[:,'sigma'] * np.sqrt(x)
    exp_upper = np.exp((table1.loc[:,'omega'] - table1.loc[:,'q']) * T + (table1.loc[:,'mu'] + table1.loc[:,'sigma'] ** 2 / 2) * x)
    X_price_numerical = ((table1.loc[:,'X_0'] * exp_upper * norm.cdf(d_1) - K_i * np.exp(-r * T) * norm.cdf(d_2)))
    S_price_numerical = sum(X_price_numerical * table1.loc[:,'w'])
    return max(S_price_numerical, 0) * gamma_density

In [ ]:
C_upper = sci.quad(f, 0, np.inf)[0]
C_upper

27.54488588725899

##### lower

In [ ]:
# 通过奇怪的K解决了解析式的问题...K_i和omega点乘=K就可以了！
def g(x):
    gamma_density = gamma(a=T/v, scale=v).pdf(x) # ((1/v) ** (T/v)) / (math.gamma(T/v)) * (x ** (T/v - 1)) * np.exp(-x / v)
    K_i = table1.loc[:,'K']
    d_1 = (np.log(table1.loc[:,'X_0']/K_i) + (r - table1.loc[:,'q'] + table1.loc[:,'omega']) * T + table1.loc[:,'mu'] * x + table1.loc[:,'sigma'] ** 2 * x)/(table1.loc[:,'sigma'] * np.sqrt(x))
    d_2 = d_1 - table1.loc[:,'sigma'] * np.sqrt(x)
    exp_upper = np.exp((table1.loc[:,'omega'] - table1.loc[:,'q']) * T + (table1.loc[:,'mu'] + table1.loc[:,'sigma'] ** 2 / 2) * x)
    X_price_numerical = ((table1.loc[:,'X_0'] * exp_upper * norm.cdf(d_1) - K_i * np.exp(-r * T) * norm.cdf(d_2)))
    S_price_numerical = sum(X_price_numerical * table1.loc[:,'w'])
    return max(S_price_numerical, 0) * gamma_density

In [ ]:
C_lower = sci.quad(g, 0, np.inf)[0]
C_lower

27.54488588725899

In [ ]:
# 看看差距
C_upper/C_sim - 1, C_upper/27.3230 - 1

(0.010346517532540123, 0.008120846439226614)

In [ ]:
# C_upper = 1
# C_lower = 1
# z_y = 1